In [1]:
%pwd

'/Users/Bixing/Google Drive (by783@nyu.edu)/Census digitization'

In [2]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import re

In [3]:
file_path='Data/Processed/'
file_1935='1935.xlsx'
file_1948='1948.xlsx'
file_1940='1940.xlsx'
couty_list_path='Data/Input/'+'ipums_usa_identified_counties.xlsx'

# Cannot fix the county with present county name list

In [4]:
df_county_list=pd.read_excel(couty_list_path,header=[0,1],skiprows=None)

In [5]:
df_county_list.head(15)

State   County Name (ICPSR) IPUMS USA Variables                    
         Unnamed: 0_level_1            STATEFIP COUNTYICP COUNTYFIP
Alabama             Baldwin                   1        30         3
Alabama      Calhoun/Benton                   1       150        15
Alabama              Dallas                   1       470        47
Alabama              Etowah                   1       550        55
Alabama           Jefferson                   1       730        73
Alabama                 Lee                   1       810        81
Alabama             Madison                   1       890        89
Alabama              Mobile                   1       970        97
Alabama          Montgomery                   1      1010       101
Alabama              Shelby                   1      1170       117
Alabama           Talladega                   1      1210       121
Alabama          Tuscaloosa                   1      1250       125
Alabama              Walker                   1      1270       127
Alaska            Anchorage                   2       200        20
Arizona            Coconino                   4        50         5

In [6]:
df_raw_1935=pd.read_excel(file_path+file_1935,header=[0,1],skiprows=None)
df_raw_1940=pd.read_excel(file_path+file_1940,header=[0,1],skiprows=None)
df_raw_1948=pd.read_excel(file_path+file_1948,header=[0,1],skiprows=None)

In [7]:
df_raw_1935.head(3)

Total          Food stores       Eating places        \
                Store Numbers  Sales Store Numbers Sales Store Numbers Sales   
Autauga County            138   1121            37   165             8    11   
Baldwin County            339   3175            95   723            22    58   
Barbour County:           249  2.843            81   724            17    41   

                General stores (with food)       General merchandise group  \
                             Store Numbers Sales             Store Numbers   
Autauga County                          44   529                         3   
Baldwin County                          46   589                        10   
Barbour County:                         35   320                        13   

                              ...         Filling stations  \
                Sales         ...                    Sales   
Autauga County     36         ...                       93   
Baldwin County     92         ...                      280   
Barbour County:   173         ...                      226   

                Furniture household radio       Lumber building hardware  \
                            Store Numbers Sales            Store Numbers   
Autauga County                          3     z                        4   
Baldwin County                         14   138                       13   
Barbour County:                         5   198                        9   

                        Drug stores        Other stores        \
                Sales Store Numbers Sales Store Numbers Sales   
Autauga County     42             5    47             4    51   
Baldwin County    285            11   129            24   168   
Barbour County:   168            11   115            15   127   

                              State  
                Unnamed: 24_level_1  
Autauga County              Alabama  
Baldwin County              Alabama  
Barbour County:             Alabama  

[3 rows x 25 columns]

In [8]:
df_raw_1940.head(3)

Location   Total           Food group          \
              County and city  Stores     Sales     Stores   Sales   
0     State Total___2,832,961  23.916  *435.973      8.802  593.91   
1    Autauga County----20,977     148   • 1,460         57     481   
2  Baldwin County------32,324     381      5018        136    1354   

  General stores (with food)          General merchandise group           \
                      Stores    Sales                    Stores    Sales   
0                      2.033  636.747                       934  655.497   
1                         20      289                         2      (x)   
2                         13      288                        18      208   

  Apparel group   ...    Grocery, combination stores (Group 1)  \
         Stores   ...                                    Sales   
0           915   ...                                    89665   
1             5   ...                                      474   
2             3   ...                                     1267   

  Hardwares stores (Group 8)       Restaurants, other eating places (Group 9)  \
                      Stores Sales                                     Stores   
0                        297  8431                                      2.145   
1                          2   (x)                                          9   
2                          8   268                                         21   

         Liquor stores (Group 11)        \
   Sales                   Stores Sales   
0  15279                       69  9120   
1     16                        —     —   
2     72                        3   (x)   

  Feed, farm, garden supplies (Group 11)       Location  
                                  Stores Sales    State  
0                                    340  9809  Alabama  
1                                      1   (x)  Alabama  
2                                     13   340  Alabama  

[3 rows x 36 columns]

In [9]:
df_raw_1948.head(3)

Location          Total                                     \
       County and city Establishments  Sales Pay roll Active proprietors   
0       Autauga County            168  5.068      224                160   
1           Prattville             82   3215      188                 82   
2  Remainder of County             86   1853       36                 78   

                                               Food group        \
  Total paid employees Full workweek employees     Stores Sales   
0                  276                     175         82  2084   
1                  229                     152         31  1037   
2                   47                      23         51  1047   

  Eating and drinking places   ...    Automotive group  \
                      Stores   ...               Sales   
0                         13   ...                 536   
1                         11   ...                 536   
2                          2   ...               • • •   

  Gasoline service stations       Lumber building, hardware group        \
                     Stores Sales                          Stores Sales   
0                        24   646                               3   353   
1                        10   178                               3   353   
2                        14   468                             ...   ...   

  Drug and proprietary stores       All other retail stores         Location  
                       Stores Sales                  Stores   Sales    State  
0                           3   172                       7  _(*) .  ALABAMA  
1                           2   (x)                       6     220  ALABAMA  
2                           1   (x)                       1     (x)  ALABAMA  

[3 rows x 28 columns]

In [10]:
df_raw_1935.columns

MultiIndex(levels=[['Apparel group', 'Automotive group', 'Drug stores', 'Eating places', 'Filling stations', 'Food stores', 'Furniture household radio', 'General merchandise group', 'General stores (with food)', 'Lumber building hardware', 'Other stores', 'State', 'Total'], ['Sales', 'Store Numbers', 'Unnamed: 24_level_1']],
           labels=[[12, 12, 5, 5, 3, 3, 8, 8, 7, 7, 0, 0, 1, 1, 4, 4, 6, 6, 9, 9, 2, 2, 10, 10, 11], [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2]])

In [11]:
# calculate the number of county in Alabama in 1935
len(df_raw_1935[df_raw_1935['State']['Unnamed: 24_level_1']=='Alabama'])

153

In [12]:
# calculate the number of county in Alabama in county list
sum(df_county_list.index=='Alabama')

13

# fix the values

In [13]:
def is_number(x):
    if pd.isnull(x): return False
    return isinstance(x, (int, float))

In [14]:
#df_raw_1935[list(df_raw_1935.columns)[0]]

In [15]:
isinstance(1,int)

True

In [16]:
def fix_object(obj):
    if pd.isnull(obj):
        return None
    elif isinstance(obj, int):
        return obj
    else:
        obj_cleaned=re.sub("[^0-9]", "", str(obj))
        # only keep the numbers in the object
        try:
            return int(obj)
        except:
            return None
        

In [17]:
def return_raw_countyname(string):
    try:
        return re.sub("[^a-zA-Z\s]", "", string)
    except:
        return None
    
def return_population(string):
    try:
        return re.sub("[^0-9]", "", string)
    except:
        return None

In [18]:
#re.sub("[^0-9]", "", 'a1b2 c3d4')

In [19]:
#re.sub("[^a-zA-Z\s]", "", 'a1b2c3d4 A5B6C7D8')

In [20]:
df_1935=df_raw_1935.copy()
df_1940=df_raw_1940.copy()
df_1948=df_raw_1948.copy()

In [21]:
list(df_1935.columns)[:-1]

[('Total', 'Store Numbers'),
 ('Total', 'Sales'),
 ('Food stores', 'Store Numbers'),
 ('Food stores', 'Sales'),
 ('Eating places', 'Store Numbers'),
 ('Eating places', 'Sales'),
 ('General stores (with food)', 'Store Numbers'),
 ('General stores (with food)', 'Sales'),
 ('General merchandise group', 'Store Numbers'),
 ('General merchandise group', 'Sales'),
 ('Apparel group', 'Store Numbers'),
 ('Apparel group', 'Sales'),
 ('Automotive group', 'Store Numbers'),
 ('Automotive group', 'Sales'),
 ('Filling stations', 'Store Numbers'),
 ('Filling stations', 'Sales'),
 ('Furniture household radio', 'Store Numbers'),
 ('Furniture household radio', 'Sales'),
 ('Lumber building hardware', 'Store Numbers'),
 ('Lumber building hardware', 'Sales'),
 ('Drug stores', 'Store Numbers'),
 ('Drug stores', 'Sales'),
 ('Other stores', 'Store Numbers'),
 ('Other stores', 'Sales')]

In [22]:
list(df_1940.columns)[1:-1]

[('Total', 'Stores'),
 ('Total', 'Sales'),
 ('Food group', 'Stores'),
 ('Food group', 'Sales'),
 ('General stores (with food)', 'Stores'),
 ('General stores (with food)', 'Sales'),
 ('General merchandise group', 'Stores'),
 ('General merchandise group', 'Sales'),
 ('Apparel group', 'Stores'),
 ('Apparel group', 'Sales'),
 ('Furniture-household-radio group', 'Stores'),
 ('Furniture-household-radio group', 'Sales'),
 ('Automotive group', 'Stores'),
 ('Automotive group', 'Sales'),
 ('Filling stations', 'Stores'),
 ('Filling stations', 'Sales'),
 ('Lumber-building-hardware', 'Stores'),
 ('Lumber-building-hardware', 'Sales'),
 ('Eating and drinking places', 'Stores'),
 ('Eating and drinking places', 'Sales'),
 ('Drug stores', 'Stores'),
 ('Drug stores', 'Sales'),
 ('Other stores', 'Stores'),
 ('Other stores', 'Sales'),
 ('Grocery, combination stores (Group 1)', 'Stores'),
 ('Grocery, combination stores (Group 1)', 'Sales'),
 ('Hardwares stores (Group 8)', 'Stores'),
 ('Hardwares stores (Gro

In [23]:
list(df_1948.columns)[1:-1]

[('Total', 'Establishments'),
 ('Total', 'Sales'),
 ('Total', 'Pay roll'),
 ('Total', 'Active proprietors'),
 ('Total', 'Total paid employees'),
 ('Total', 'Full workweek employees'),
 ('Food group', 'Stores'),
 ('Food group', 'Sales'),
 ('Eating and drinking places', 'Stores'),
 ('Eating and drinking places', 'Sales'),
 ('General merchandise group, general stores', 'Stores'),
 ('General merchandise group, general stores', 'Sales'),
 ('Apparel group', 'Stores'),
 ('Apparel group', 'Sales'),
 ('Furniture furnishings, appliance group', 'Stores'),
 ('Furniture furnishings, appliance group', 'Sales'),
 ('Automotive group', 'Stores'),
 ('Automotive group', 'Sales'),
 ('Gasoline service stations', 'Stores'),
 ('Gasoline service stations', 'Sales'),
 ('Lumber building, hardware group', 'Stores'),
 ('Lumber building, hardware group', 'Sales'),
 ('Drug and proprietary stores', 'Stores'),
 ('Drug and proprietary stores', 'Sales'),
 ('All other retail stores', 'Stores'),
 ('All other retail store

In [24]:
for col in list(df_raw_1935.columns)[:-1]:
    df_1935[col]=list(map(fix_object,df_raw_1935[col]))

In [25]:
for col in list(df_raw_1940.columns)[1:-1]:
    df_1940[col]=list(map(fix_object,df_raw_1940[col]))

In [26]:
for col in list(df_raw_1948.columns)[1:-1]:
    df_1948[col]=list(map(fix_object,df_raw_1948[col]))

In [27]:
raw_countyidx_1940=(df_raw_1940.columns)[0]

In [28]:
df_1940[raw_countyidx_1940]=list( map( return_raw_countyname, df_raw_1940[raw_countyidx_1940] ))

In [29]:
df_1940[('population','population')]=list( map( return_population, df_raw_1940[raw_countyidx_1940] ))

In [30]:
df_1948

Location          Total  \
                                        County and city Establishments   
0                                        Autauga County          168.0   
1                                            Prattville           82.0   
2                                   Remainder of County           86.0   
3                                        Baldwin County          507.0   
4                                           Bay Minette           71.0   
5                                              Fairhope           65.0   
6                                   Remainder of County          371.0   
7                                        Barbour County          315.0   
8                                Eufaula..............1          115.0   
9                                   Remainder of County          200.0   
10                                          Bibb County          149.0   
11                                        Blount County          254.0   
12                                              Oneonta           89.0   
13                                  Remainder of County          165.0   
14                                       Bullock County          160.0   
15                                        Union Springs           79.0   
16                                  Remainder of County           81.0   
17                                        Butler County          286.0   
18                                           Greenville          124.0   
19                                  Pemainder of County          162.0   
20                                       Calhoun County          789.0   
21                                             Anniston          424.0   
22                                         Jacksonville           58.0   
23                                             Piedmont           83.0   
24                                  Remainder of County          224.0   
25                                      Chambers County          315.0   
26                                               Lanett           62.0   
27                                  Remainder of County          253.0   
28                                      Cherokee County          145.0   
29                                       Chilton County          269.0   
...                                                 ...            ...   
9336                                       Uinta County          124.0   
9337                                           Evanston            NaN   
9338                                Remainder of County           53.0   
9339                                    Washakie County           87.0   
9340                                            Worland           71.0   
9341                                Remainder of County           16.0   
9342  Weston County............. Yellowstone Nationa...           70.0   
9343                                            (part)2           20.0   
9344                            First Judicial Division          445.0   
9345                                             Juneau          116.0   
9346                                          Ketchikan          128.0   
9347                              Remainder of Division          201.0   
9348                           Second Judicial Division           90.0   
9349                            Third judicial Division          492.0   
9350                                          Anchorage          208.0   
9351                              Remainder of Division          284.0   
9352                           Fourth Judicial Division          284.0   
9353                                          Fairbanks          121.0   
9354                             Remainder of Division—          163.0   
9355                                      Hawaii County          755.0   
9356                                               Hilo          335.0   
9357                                Remainder of Co

In [31]:
file_path='./Data/Processed/'
writer = ExcelWriter(file_path+str(1935)+'_cleaned_190304.xlsx')
df_1935.to_excel(writer)
writer.save()

writer = ExcelWriter(file_path+str(1940)+'_cleaned_190304.xlsx')
df_1940.to_excel(writer)
writer.save()

writer = ExcelWriter(file_path+str(1948)+'_cleaned_190304.xlsx')
df_1948.to_excel(writer)
writer.save()